# Testing differences between groups

In [ ]:
# Import numerical, data and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Only show 4 decimals when printing
np.set_printoptions(precision=4)

In [ ]:
# Show the plots in the notebook
%matplotlib inline

Imagine we have some some measures of psychopathy in 12 students.  4 students are from Berkeley, and 4 students are from MIT.

In [ ]:
psychos = pd.read_csv('psycho_students.csv')
psychos

We find that the mean score for the Berkeley students is different from the mean score for the MIT students:

In [ ]:
berkeley_students = psychos[psychos['university'] == 'Berkeley']
berkeley_students

In [ ]:
mit_students = psychos[psychos['university'] == 'MIT']
mit_students

In [ ]:
berkeley_scores = berkeley_students['psychopathy']
mit_scores = mit_students['psychopathy']
berkeley_scores.mean(), mit_scores.mean()

Here is the difference between the means:

In [ ]:
mean_diff = berkeley_scores.mean() - mit_scores.mean()
mean_diff

That's the difference we see.  But - if we take any 8 students from a single university, and take the mean of the first four, and the mean of the second four, there will almost certainly be a difference in the means, just because there's some difference across individuals in the psychopathy score.  Is this difference we see unusual compared to the differences we would see if we took eight students from the same university, and compared the means of the first four and the second four?

For a moment, let us pretend that all our Berkeley and MIT students come from the same university.   Then I can pool the Berkely and MIT students together.

In [ ]:
pooled = pd.concat([berkeley_scores, mit_scores]).values
pooled

If there is no difference between Berkeley and MIT, then it should be OK to just shuffle the students to a random order, like this:

In [ ]:
np.random.shuffle(pooled)
pooled

Now I can just pretend that the first four students are from one university, and the last four are from another university.  Then I can compare the means.

In [ ]:
fake_berkeley = pooled[:4]
fake_mit = pooled[4:]
np.mean(fake_berkeley) - np.mean(fake_mit)

In [ ]:
fake_differences = np.zeros(10000)
for i in range(10000):
    np.random.shuffle(pooled)
    diff = np.mean(pooled[:4]) - np.mean(pooled[4:])
    fake_differences[i] = diff

The 10000 values we calculated form the *sampling distribution*.  Let's have a look:

In [ ]:
plt.hist(fake_differences)
plt.title("Sampling distribution of mean difference");

Where does the value we actually see, sit in this histogram? More specifically, how many of the values in this histogram are less then or equal to the value we actually see?

In [ ]:
# We will count the number of fake_differences <= our observed
count = 0
# Go through each of the 10000 values one by one
for diff in fake_differences:
    if diff <= mean_diff:
        count = count + 1
proportion = count / 10000
proportion

That's the p value.